In [ ]:
using Pkg
Pkg.add("CSV")
Pkg.add("Plots")
Pkg.add("Statistics")
Pkg.add("DataFrames")
Pkg.add("Query")
Pkg.build("CodecZlib")
using CSV
using DataFrames
using Query
using Plots
using Statistics


## Data Set de AUSA
    - Pasos de autos por los distintos puestos de peaje
    - Años: 2015, 2016, 2017, 2018
    - 3MM de registros
### Preguntas
    - ¿ Cuál es el peor dia para ir por las autopistas de AUSA ?
    - ¿ Cuál es la peor hora ?
### Muestra del Data Set
~~~~
periodo,fecha,dia,hora,hora_fin,estacion,tipo_vehiculo,forma_pago,cantidad_pasos
2018,2018-01-01,LUNES,00:00:00,01:00:00,ALBERDI,LIVIANO,NO COBRADO,75
2018,2018-01-01,LUNES,00:00:00,01:00:00,ALBERDI,LIVIANO,AUPASS,24
~~~~

In [ ]:
function normalize(df::DataFrame)
    x = @from i in df begin
        @select {periodo = i.periodo,
        fecha=i.fecha,
        dia=lowercase(i.dia),
        hora_desde=i.hora,
        hora_hasta=i.hora_fin,
        estacion=i.estacion,
        tipo_vehiculo=lowercase(i.tipo_vehiculo),
        forma_pago=i.forma_pago,
        cantidad_pasos=i.cantidad_pasos
        }
        @collect DataFrame
    end
    return x
end


In [ ]:
flujo = "../examples/datos/flujo-vehicular-all-v0.1.csv"
df = CSV.File(flujo) |> DataFrame
df = normalize(df)
head(df)

### Totales por dia

In [ ]:
head(by(df, :dia, size))

In [ ]:
# suma de cantidad_pasos por dia
pasos_dia = by(df, :dia, d -> DataFrame(pasos = sum(d.cantidad_pasos)))
pasos_dia = sort(pasos_dia,:pasos)

# dibujar mas copado
plot(size=(800,800),leg=false)
x = pasos_dia.dia
y = pasos_dia.pasos
scatter(x,y,markersize=5)